## LangChain 経由で Amazon Kendra を使った RAG を実装する

![architecture.png](./figs/architecture.png)

こちらのサンプルでは、AWS でのインテリジェントな検索を実現する Amazon Kendra を使った検索拡張生成（RAG, Retrieval Augmented Generation）の実現方法を解説します。  

LangChain 経由で SageMaker を使う例は [LangChain 経由で SageMaker でホストした大規模言語モデル (LLM) を使う (Notebook)](./langchain-sagemaker-intro.ipynb) で紹介しているので、SageMaker 上で立ち上げた大規模言語モデル（LLM）の利用方法の詳細についてはこちらを参照してください。  

別のサンプルと同様に、HuggingFace 上で rinna 社が公開している [rinna/japanese-gpt-neox-3.6b-instruction-ppo](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo) を使用します。  

こちらの Notebook は以下の環境で動作確認を行なっています。

- SageMaker Studio Notebooks
    - `ml.t3.medium`: `Data Science 3.0`
- SageMaker Notebooks
    - `ml.t3.medium`: `conda_python3`

[各インスタンスの料金についてはこちら](https://aws.amazon.com/jp/sagemaker/pricing/)をご確認ください。  
 



## Amazon Kendra とは

Amazon Kendra は、機械学習を活用したマネージドな検索サービスです。設定手順も簡素化されており誰でも簡単に検索システムを構築可能です。  

Amazon Kendra では主に以下のタイプの質問および検索をサポートしています。  
- Factoid 型の質問: 「Kendra が一般利用可能になったのはいつですか？」などの誰が、何を、いつ、どこで、といったことを問う質問
- Non-Factoid 型の質問: 「Kendra はどのようなサービスですか？」などの理由や事象の説明に基づく回答を求める質問
- キーワードまたは自然言語による検索: 「Kendra チュートリアル」や「Kendra のチュートリアル」といった検索



## 前準備

事前準備として LLM を SageMaker Realtime Endpoint でホストします。  
下記の Notebook を実行することでエンドポイントを立てます。  
`git pull https://github.com/aws-samples/aws-ml-jp.git` などでこのサンプルコードをダウンロードしてきている場合は `tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/Transformers/Rinna_Neox_Inference_ja.ipynb` の path を参照してください。  
https://github.com/aws-samples/aws-ml-jp/blob/main/tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/Transformers/Rinna_Neox_Inference_ja.ipynb


In [ ]:
# endpoint_name = <エンドポイント名>
endpoint_name = "Rinna-Inference"

#### Kendra Index の作成

今回はユーザーからの質問に対して、Kendra からの検索結果を組み合わせる形で LLM に作文してもらいます。  
そのためにはまず Kendra に検索対象となるドキュメントを登録します。今回は下記のブログの手順に従ってインデックスを作成した前提で進めていきます。  
- [Amazon Kendra で簡単に検索システムを作ってみよう ! - 変化を求めるデベロッパーを応援するウェブマガジン | AWS](https://aws.amazon.com/jp/builders-flash/202302/kendra-search-system)  

他にも[AWS ドキュメント (Getting started with the Amazon Kendra console)](https://docs.aws.amazon.com/ja_jp/kendra/latest/dg/gs-console.html)や[Simple Lex Kendra JP (サンプルプロジェクト)](https://github.com/aws-samples/simple-lex-kendra-jp)などが参考になります。   

作成した Kendra インデックスの ID を下記のパラメータに置き換えます。  

![kendra-index](./figs/kendra-index.png)

In [ ]:
kendra_index_id = <kendra_index_id> # ここの値を置き換える

### 必要権限の設定

今回のサンプルでは Kendra に対しても検索クエリが叩けるような権限を付与する必要があります。  
下記の `<region_name>` と `<account_id>` と `<kendra_index_id>` を適宜書き換える形でポリシーを作成し SageMaker Studio もしくは SageMaker Notebook インスタンスの実行ロールにアタッチします。  

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "kendra:Query",
                "kendra:Retrieve"
            ],
            "Resource": "arn:aws:kendra:<region_name>:<account_id>:index/<kendra_index_id>"
        }
    ]
}
```

詳しくは[こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください。

### 必要モジュールのインストール

まずは、必要モジュールのインストールをします。

In [ ]:
!pip install 'langchain>=0.0.215'
!pip install -U sagemaker boto3

#### 必要モジュールのインストール

今回のサンプルで使用する必要モジュールのインストールをしていきます。  
Kendra の利用には LangChain で提供されている AmazonKendraRetriever を使用します。  

In [ ]:
import sys
import json
import os
from typing import Dict

import boto3
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.retrievers import AmazonKendraRetriever

#### Kendra Retriever の作成

In [ ]:
region_name = "us-east-1"  # 今使用しているリージョンに置き換えてください

Kendra Client の設定をします。ここで検索結果のフィルタリングを行う際の一般的な設定も指定します。  
ここでは例として言語コードを日本語である `ja` に指定することで日本語として登録されたデータソースに対してクエリを叩くよう設定しています。

In [ ]:
kendra_client = boto3.client("kendra", region_name=region_name)
language_code = "ja"
retriever = AmazonKendraRetriever(
    client=kendra_client,
    index_id=kendra_index_id,
    attribute_filter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {"StringValue": language_code},
        }
    }
)

retriever.get_relevant_documents を呼び出すと Kendra に対して検索を実施できます。

In [ ]:
retriever.get_relevant_documents("Lambda関数で使用できるメモリの最大値は？")

### Kendra を使った Chain を作成する

ここからは LangChain を使って Kendra からの検索結果を使った応答を SageMaker 上にホストした LLM にやらせてみます。

SageMaker 周りのコードの詳細ついてはサンプル [LangChain 経由で SageMaker でホストした大規模言語モデル (LLM) を使う](https://github.com/aws-samples/aws-ml-jp/blob/main/tasks/generative-ai/text-to-text/inference/langchain/langchain-sagemaker-intro.ipynb)で解説しているので適宜参照してください。  

本サンプルでは Kendra に関係するコードの解説を中心的に行います。  

以下のセルは、Kendra と SageMaker を使って、質問応答（Q&A）システムを構築しています。ここでは、チャット形式の対話を行うために、言語モデルと情報検索モデルを組み合わせた「Retrieval Chain」（情報検索チェーン）を構築します。

In [ ]:
kendra_client = boto3.client("kendra", region_name=region_name)


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps(
            {
                "input": prompt.replace("\n", "<NL>"), 
                "instruction": "", 
                **model_kwargs
            })
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json.replace("<NL>", "\n")

def build_chain(prompt: PromptTemplate) -> RetrievalQA:
    """Kendra を使った Retrieval Chain の構築
    """
    content_handler = ContentHandler()
    llm = SagemakerEndpoint(
            endpoint_name=endpoint_name, 
            region_name=region_name, 
            model_kwargs={
            "max_new_tokens": 128,
            "temperature": 0.7,
            "do_sample": True,
            "pad_token_id": 0,
            "bos_token_id": 2,
            "eos_token_id": 3, #265,  # 「。」の ID に相当。
            "stop_ids": [50278, 50279, 50277, 1, 0],
        },
            content_handler=content_handler
        )
    language_code = "ja"
    # Amazon Kendra Retriever を設定します
    retriever = AmazonKendraRetriever(
        client=kendra_client,
        index_id=kendra_index_id,
        top_k=2,
        attribute_filter={
            "EqualsTo": {
                "Key": "_language_code",
                "Value": {"StringValue": language_code},
            }
        }
    )
    chain_type_kwargs = {
        "prompt": prompt
    }  
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs = chain_type_kwargs,
        return_source_documents=True,
        verbose=True
    )
    return qa


def run_chain(chain, prompt: str):
    """構築した chain を実行する関数
    """
    return chain(prompt)



### Chain を呼び出す

ここから定義した chain を作成して実際に呼び出してみます。  

そのために、Kendra からの検索結果を踏まえた上で LLM に要約の指示を出すプロンプトを指定します。  
（こちらのプロンプトはまだベストに動いているものを模索しきれていないので、ぜひ皆さんで工夫してもっと良く動くものあればぜひ Pull Request をお願いします！）

In [ ]:
prompt_template ="""
システム: 以下は、ユーザーとシステムとの会話です。システムは資料から抜粋して質問に答えます。資料にない内容は答えず「わかりません」と答えます。

{context}

上記の資料に基づき以下の質問について資料から抜粋して回答してください。資料にない内容は答えず「わかりません」と答えてください。

ユーザー: {question}
システム:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chat_history=[]
chain = build_chain(prompt)

では実際にクエリを叩いてみましょう

In [ ]:
query = "Lambdaで利用できるメモリの最大値は？"
result = run_chain(chain, query)
print(result["result"])

LLM によって要約された結果を確認することができました。